<a href="https://colab.research.google.com/github/FelipeSotoG/Desafio-Final/blob/main/Desafio_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boxing usando Actor Crítico (PPO)
Basado en [este algoritmo](https://keras.io/examples/rl/ppo_cartpole/)

Instalamos paquetes necesarios

In [1]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=9665681c5f9ca458b3a0f44b1f333aa47ab2079b8c309528e6985fad1c95faf6
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license


In [2]:
!mv /usr/local/lib/python3.7/dist-packages/AutoROM/roms/boxing.bin /content

In [3]:
!pip install gym-retro > /dev/null 2>&1 
!python3 -m retro.import .  > /dev/null 2>&1 

In [ ]:
!pip install ale-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 13.2 MB/s 


In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common > /dev/null 2>&1 

!apt-get install -y patchelf > /dev/null 2>&1 

In [ ]:
# Librerías para el entorno de gym
!pip install colabgymrender > /dev/null 2>&1 
!pip install free-mujoco-py > /dev/null 2>&1 

## Librerías necesarias

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import gym
import scipy.signal
import time

## Parámetros del algoritmo

In [48]:
n_frames= 10
steps_per_epoch = 1000
epochs = 250
ratio = 0.1
hidden_sizes = (64, 64)
policy_learning_rate = 3e-4
value_function_learning_rate = 1e-3
train_policy_iterations = 80
train_value_iterations = 80
clip_ratio = 0.2
gamma = 0.99
lam = 0.97
target_kl = 0.01


se uso 1000 steps y 10 epochs por razones de tiempo

#Base

## Buffer para almacenar las muestras

In [ ]:
class Buffer:
    # Buffer for storing trajectories
    def __init__(self, observation_dimensions, size, gamma=0.99, lam=0.95):
        # Buffer initialization
        self.observation_buffer = np.zeros(
            (size, observation_dimensions), dtype=np.float32
        )
        self.action_buffer = np.zeros(size, dtype=np.int32)
        self.advantage_buffer = np.zeros(size, dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.return_buffer = np.zeros(size, dtype=np.float32)
        self.value_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.pointer, self.trajectory_start_index = 0, 0

    def store(self, observation, action, reward, value, logprobability):
        # Append one step of agent-environment interaction
        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.value_buffer[self.pointer] = value
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1

    def finish_trajectory(self, last_value=0):
        # Finish the trajectory by computing advantage estimates and rewards-to-go
        path_slice = slice(self.trajectory_start_index, self.pointer)
        rewards = np.append(self.reward_buffer[path_slice], last_value)
        values = np.append(self.value_buffer[path_slice], last_value)

        deltas = rewards[:-1] + self.gamma * values[1:] - values[:-1]

        self.advantage_buffer[path_slice] = discounted_cumulative_sums(
            deltas, self.gamma * self.lam
        )
        self.return_buffer[path_slice] = discounted_cumulative_sums(
            rewards, self.gamma
        )[:-1]

        self.trajectory_start_index = self.pointer

    def get(self):
        # Get all data of the buffer and normalize the advantages
        self.pointer, self.trajectory_start_index = 0, 0
        advantage_mean, advantage_std = (
            np.mean(self.advantage_buffer),
            np.std(self.advantage_buffer),
        )
        self.advantage_buffer = (self.advantage_buffer - advantage_mean) / advantage_std
        return (
            self.observation_buffer,
            self.action_buffer,
            self.advantage_buffer,
            self.return_buffer,
            self.logprobability_buffer,
        )

## Creamos los modelos para el actor y el crítico

In [ ]:
import retro
# Initialize the environment and get the dimensionality of the
# observation space and the number of possible actions
# Create our environment

if 'env' in locals(): env.close()

env = gym.make('Boxing-v4')
#env = retro.make(game='SpaceInvaders-Atari2600')
#env = retro.make(game='Boxing-Atari2600')
#observation_dimensions = 14*n_frames # 14 objects
observation_dimensions = 122880
num_actions = env.action_space.n
possible_actions = np.array(np.identity(num_actions,dtype=int).tolist())


print("The size of our frame is: ", env.observation_space)

# Here we create an hot encoded version of our actions
# possible_actions = [[1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]...]
possible_actions = np.array(np.identity(num_actions,dtype=int).tolist())


def mlp(x, sizes, activation=tf.tanh, output_activation=None):    
    # Build a feedforward neural network
    for size in sizes[:-1]:
        x = layers.Dense(units=size, activation=activation)(x)
    return layers.Dense(units=sizes[-1], activation=output_activation)(x)

# Train the policy by maxizing the PPO-Clip objective
@tf.function
def train_policy(
    observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
):

    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        ratio = tf.exp(
            logprobabilities(actor(observation_buffer), action_buffer)
            - logprobability_buffer
        )
        min_advantage = tf.where(
            advantage_buffer > 0,
            (1 + clip_ratio) * advantage_buffer,
            (1 - clip_ratio) * advantage_buffer,
        )

        policy_loss = -tf.reduce_mean(
            tf.minimum(ratio * advantage_buffer, min_advantage)
        )
    policy_grads = tape.gradient(policy_loss, actor.trainable_variables)
    policy_optimizer.apply_gradients(zip(policy_grads, actor.trainable_variables))

    kl = tf.reduce_mean(
        logprobability_buffer
        - logprobabilities(actor(observation_buffer), action_buffer)
    )
    kl = tf.reduce_sum(kl)
    return kl


# Train the value function by regression on mean-squared error
@tf.function
def train_value_function(observation_buffer, return_buffer):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        value_loss = tf.reduce_mean((return_buffer - critic(observation_buffer)) ** 2)
    value_grads = tape.gradient(value_loss, critic.trainable_variables)
    value_optimizer.apply_gradients(zip(value_grads, critic.trainable_variables))

# Initialize the actor and the critic as keras models
observation_input = keras.Input(shape=(observation_dimensions,), dtype=tf.float32)
logits = mlp(observation_input, list(hidden_sizes) + [num_actions], tf.tanh, None)
actor = keras.Model(inputs=observation_input, outputs=logits)
value = tf.squeeze(
    mlp(observation_input, list(hidden_sizes) + [1], tf.tanh, None), axis=1
)
critic = keras.Model(inputs=observation_input, outputs=value)

# Initialize the policy and the value function optimizers
policy_optimizer = keras.optimizers.Adam(learning_rate=policy_learning_rate)
value_optimizer = keras.optimizers.Adam(learning_rate=value_function_learning_rate)

The size of our frame is:  Box(0, 255, (210, 160, 3), uint8)


In [ ]:
retro.data.list_games()

In [ ]:
def discounted_cumulative_sums(x, discount):
    # Discounted cumulative sums of vectors for computing rewards-to-go and advantage estimates
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]



def logprobabilities(logits, a):
    # Compute the log-probabilities of taking actions a by using the logits (i.e. the output of the actor)
    logprobabilities_all = tf.nn.log_softmax(logits)
    logprobability = tf.reduce_sum(
        tf.one_hot(a, num_actions) * logprobabilities_all, axis=1
    )
    return logprobability


# Sample action from actor
@tf.function
def sample_action(observation):
    logits = actor(observation)
    action = tf.squeeze(tf.random.categorical(logits, 1), axis=1)
    return logits, action

## Extractor de objetos

La función `image2state` es la encargada de convertir un estado (imagen) en un vector de posiciones de objetos.

![image](https://i.imgur.com/fYygQNe.png)

## Función para "stackear" frames

Para que el agente entienda la velocidad y movimiento de los objetos, consideramos que un estado considera un **conjunto de observaciones** y no sólo la última retornada por el ambiente.

La función `stacked_state` mantiene un estado compuesto por los últimos `n_frames=4` estados. Este será el estado que observe el agente.

In [ ]:
from collections import deque

stacked_frames = None

def stacked_state(state, is_new_episode):  
    global stacked_frames
    if is_new_episode or stacked_frames==None:

        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((observation_dimensions), dtype=np.int) for i in range(n_frames)], maxlen=n_frames)  
        # Because we're in a new episode, copy the same frame 4x
        for i in range(n_frames):
          stacked_frames.append(state)
        
        # Stack the state
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        # Append frame to deque, automatically removes the oldest frame
        if state is not None: stacked_frames.append(state)
        else: stacked_frames.append(np.zeros((observation_dimensions), dtype=np.int))

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state

## Funciones para interactuar con ambiente

Se encapsulan las funciones del ambiente para realizar un par de modificaciones:

- `env_step`: Permite repetir la acción por `iters` frames y retorna el stacked state resultante junto a la recompensa acumulada de los 5 frames. Esto permite que los episodios se realicen más rápidamente. Además, el episodio termina al perder la primera vida.

- `env_reset`: Además de resetear el ambiente, avanza 130 iteraciones "saltándose" la introducción del juego.

Se eligen modelos de extraccion de caracteristicas, los cuales se probaran para su efectividad en la busqueda PPO

Estas caracteristicas se usaran en vez de la posicion de los items del juego para entrenar al modelo.

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet import ResNet50

In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(210, 160, 3))
for layer in VGG_model.layers:
	layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
VGG_model = MobileNet(weights='imagenet', include_top=False, input_shape=(210, 160, 3), classes=2)
for layer in VGG_model.layers:
	layer.trainable = False

In [ ]:
VGG_model = MobileNet(weights='imagenet', include_top=False, input_shape=(210, 160, 3))
for layer in VGG_model.layers:
	layer.trainable = False

In [ ]:
VGG_model = ResNet50(weights='imagenet', include_top=False, input_shape=(210, 160, 3), classes=6)
for layer in VGG_model.layers:
	layer.trainable = False

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
def env_reset(env):
  env.reset()
  state, reward, done, _ = env_step(env, 0, iters=130, new_episode=True) #starting
  return state

images = None
def env_step(env, action, iters=5, new_episode=False, show=False):
  reward=0.
  action = possible_actions[action][action]
  if iters==1:
    obs, r, done, info = env.step(action)
    obs=obs.reshape((1,210,160,3))
    next_state = VGG_model.predict(obs)
    return next_state, r, done, _
  for i in range(iters):
    obs, r, done, info = env.step(action)
    reward+=r
    if done: break
  if(show): images.append(obs)
  obs=obs.reshape((1,210,160,3))
  next_state = VGG_model.predict(obs) ## arreglo de posiciones
  
  state = stacked_state(next_state,new_episode)

  return state, reward, done, _

In [ ]:
env.reset()
state, reward, done, info = env_step(env, 0, iters=130, new_episode=True)

In [ ]:
print(done)
print(info)

False



## Entrenamiento (AC3)

El entrenamiento consiste en lo siguiente:

- Se recolectan datos para entrenar modelos
<estado, acción, recompensa>. Estos datos son guardados en el buffer (línea 29)

- Para la recolección de datos, se sigue política softmax usando predicciones del actor (línea 17).

- Al terminar un episodio, se calculan retornos y advantages para cada estado. Advantages son calculados usando crítico (función `finish_trajectory`, línea 38).

- Se ajustan los modelos (líneas 55-66):
  
  - Crítico predice retorno esperado a partir de cada estado. 
  - Actor predice advantages para cada estado-acción.

- Se repite todo el proceso.

![image](https://i.imgur.com/rd5tda1.png)

In [ ]:
# Initialize the buffer
buffer = Buffer(observation_dimensions, steps_per_epoch)

# Initialize the observation, episode return and episode length
observation, episode_return, episode_length = env_reset(env), 0, 0

# Iterate over the number of epochs
for epoch in range(epochs):
    # Initialize the sum of the returns, lengths and number of episodes for each epoch
    sum_return = 0;   sum_length = 0;   num_episodes = 0;   episode_max_return = 0

    # Iterate over the steps of each epoch
    for t in range(steps_per_epoch):
        # Get the logits, action, and take one step in the environment
        observation = observation.reshape(1, -1)
        
        logits, action = sample_action(observation)
        observation_new, reward, done, _ = env_step(env, action[0].numpy(), iters=5)
        episode_return += reward
        episode_length += 1

        # Get the value and log-probability of the action
        value_t = critic(observation)
        logprobability_t = logprobabilities(logits, action)

        # Store obs, act, rew, v_t, logp_pi_t
        buffer.store(observation, action, reward, value_t, logprobability_t)

        # Update the observation
        observation = observation_new

        # Finish trajectory if reached to a terminal state
        terminal = done
        if terminal or (t == steps_per_epoch - 1):
            last_value = 0 if done else critic(observation.reshape(1, -1))
            buffer.finish_trajectory(last_value)
            sum_return += episode_return
            sum_length += episode_length
            num_episodes += 1
            if episode_return > episode_max_return: episode_max_return=episode_return
            observation, episode_return, episode_length = env_reset(env), 0, 0 
            
    # Get values from the buffer
    (
        observation_buffer,
        action_buffer,
        advantage_buffer,
        return_buffer,
        logprobability_buffer,
    ) = buffer.get()

    # Update the policy and implement early stopping using KL divergence
    for _ in range(train_policy_iterations):
        kl = train_policy(
            observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
        )
        if kl > 1.5 * target_kl:
            # Early Stopping
            break

    # Update the value function
    for _ in range(train_value_iterations):
        train_value_function(observation_buffer, return_buffer)

    # Print mean return and length for each epoch
    print(
        f" Epoch: {epoch + 1}. MeanReturn: {sum_return / num_episodes}. MeanLength: {sum_length / num_episodes}. MaxReturn: {episode_max_return}"
    )

 Epoch: 1. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 2. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 3. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 4. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 5. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 6. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 7. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 8. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 9. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0
 Epoch: 10. MeanReturn: -22.5. MeanLength: 250.0. MaxReturn: 0


### Simulación

In [ ]:

episode_return=0.0
i=0
while episode_return < 500 and i<10:
  observation, episode_return, episode_length = env_reset(env), 0, 0
  images = []
  for t in range(steps_per_epoch):
      # Get the logits, action, and take one step in the environment
      observation = observation.reshape(1, -1)
      
      logits, action = sample_action(observation)
      observation_new, reward, done, _ = env_step(env, action[0].numpy(), iters=5, show=True)

      episode_return += reward

      # Update the observation
      observation = observation_new

      # Finish trajectory if reached to a terminal state
      if done:
        observation = env_reset(env)
        print(episode_return)
        break
  i+=1
episode_return

-30.0
-30.0
-30.0
-30.0
-30.0
-30.0
-30.0
-30.0
-30.0
-30.0


-30.0

### Creación del video

In [ ]:
# Librearías requeridas
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

##Space invader 10 epocas y 1000 iteraciones Mobile Net

In [ ]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 40.0, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400)

100%|██████████| 862/862 [00:00<00:00, 1270.12it/s]


## Boxing v4 10 epocas y 1000 ireraciones Mobile Net

In [ ]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 10, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400)

100%|█████████▉| 332/333 [00:00<00:00, 751.77it/s]


Con el cambio de stack frames

In [ ]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 20, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400)

Fallo por como funciona el stacking de frames. porque no tenia suficientes frames para la cantidad de operaciones. Por lo que no llegaba a un estado "done", porque tenia que pasar 2 minutos.

#Modelo con convoluciones

Step para convoluciones

In [35]:
def env_reset(env, iter=5):
  env.reset()
  state, reward, done, _ = env_step(env, 0, iters=iter, new_episode=True) #starting
  return state

images = None
import tensorflow as tf
def env_step(env, action, iters=5, new_episode=False, show=False):
  reward=0.
  action = possible_actions[action]
  for i in range(iters):
    obs, r, done, info = env.step(action)
    obs=tf.image.resize(obs,(100,80))
    reward+=r
    if done: break
  if(show): images.append(obs)
  
  state = stacked_state(obs,new_episode)

  return state, reward, done, _

Buffer para convolucion

In [7]:
class Buffer:
    # Buffer for storing trajectories
    def __init__(self, observation_dimensions, size, gamma=0.99, lam=0.95):
        # Buffer initialization
        self.observation_buffer = np.zeros(
            (size,observation_dimensions[0],observation_dimensions[1],observation_dimensions[2]), dtype=np.float32
        )
        self.action_buffer = np.zeros(size, dtype=np.int32)
        self.advantage_buffer = np.zeros(size, dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.return_buffer = np.zeros(size, dtype=np.float32)
        self.value_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.pointer, self.trajectory_start_index = 0, 0

    def store(self, observation, action, reward, value, logprobability):
        # Append one step of agent-environment interaction
        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.value_buffer[self.pointer] = value
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1

    def finish_trajectory(self, last_value=0):
        # Finish the trajectory by computing advantage estimates and rewards-to-go
        path_slice = slice(self.trajectory_start_index, self.pointer)
        rewards = np.append(self.reward_buffer[path_slice], last_value)
        values = np.append(self.value_buffer[path_slice], last_value)

        deltas = rewards[:-1] + self.gamma * values[1:] - values[:-1]

        self.advantage_buffer[path_slice] = discounted_cumulative_sums(
            deltas, self.gamma * self.lam
        )
        self.return_buffer[path_slice] = discounted_cumulative_sums(
            rewards, self.gamma
        )[:-1]

        self.trajectory_start_index = self.pointer

    def get(self):
        # Get all data of the buffer and normalize the advantages
        self.pointer, self.trajectory_start_index = 0, 0
        advantage_mean, advantage_std = (
            np.mean(self.advantage_buffer),
            np.std(self.advantage_buffer),
        )
        self.advantage_buffer = (self.advantage_buffer - advantage_mean) / advantage_std
        return (
            self.observation_buffer,
            self.action_buffer,
            self.advantage_buffer,
            self.return_buffer,
            self.logprobability_buffer,
        )

In [8]:
def discounted_cumulative_sums(x, discount):
    # Discounted cumulative sums of vectors for computing rewards-to-go and advantage estimates
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]



def logprobabilities(logits, a):
    # Compute the log-probabilities of taking actions a by using the logits (i.e. the output of the actor)
    logprobabilities_all = tf.nn.log_softmax(logits)
    logprobability = tf.reduce_sum(
        tf.one_hot(a, num_actions) * logprobabilities_all, axis=1
    )
    return logprobability


# Sample action from actor
@tf.function
def sample_action(observation):
    logits = actor(observation)
    action = tf.squeeze(tf.random.categorical(logits, 1), axis=1)
    return logits, action

stack para Convoluciones

In [9]:
from collections import deque

stacked_frames = None

def stacked_state(state, is_new_episode):  
    global stacked_frames
    if is_new_episode or stacked_frames==None:

        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((observation_dimensions), dtype=np.int) for i in range(n_frames)], maxlen=n_frames)  
        # Because we're in a new episode, copy the same frame 4x
        for i in range(n_frames):
          stacked_frames.append(state)
        
        # Stack the state
        stacked_state = np.concatenate(stacked_frames, axis=2)
        
    else:
        # Append frame to deque, automatically removes the oldest frame
        if state is not None: stacked_frames.append(state)
        else: stacked_frames.append(np.zeros(observation_dimensions, dtype=np.int))

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.concatenate(stacked_frames, axis=2) 
    
    return stacked_state

Convolution version

Este es un bloque de convolucion basado en Resnet

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.jeremyjordan.me%2Fcontent%2Fimages%2F2018%2F04%2FScreen-Shot-2018-04-16-at-5.47.38-PM.png&f=1&nofb=1)" width=500px>

In [10]:
def conv_block(X,f,d=0.1):
  c = tf.keras.layers.Conv2D(f[0], (1, 1), activation='relu', kernel_initializer='he_normal', padding='same')(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[2], (1, 1), kernel_initializer='he_normal', padding='same')(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  s = tf.keras.layers.Conv2D(f[2], (1, 1), kernel_initializer='he_normal', padding='same')(X)
  s = tf.keras.layers.BatchNormalization(axis=3)(s)
  c = tf.keras.layers.Add()([s,c])
  c = tf.keras.layers.ReLU()(c)
  return c

Este es un bloque de convolucion basado en Unet
![image](https://www.researchgate.net/publication/341300084/figure/fig4/AS:890174032580608@1589245436074/The-proposed-U-net-architecture.png)

In [ ]:
def conv_block(X,f,d=0.1):
  c = tf.keras.layers.Conv2D(f[0], (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  return c

In [24]:
if 'env' in locals(): env.close()
env = retro.make(game='Boxing-Atari2600')
print("The size of our frame is: ", env.observation_space.shape)

The size of our frame is:  (210, 160, 3)


In [44]:
import gym
import retro
# Initialize the environment and get the dimensionality of the
# observation space and the number of possible actions
# Create our environment

if 'env' in locals(): env.close()
size=env.observation_space.shape
#env = gym.make('Boxing-v4')
#env = gym.make('ALE/Boxing-v5')
#env = retro.make(game='SpaceInvaders-Atari2600')
env = retro.make(game='Boxing-Atari2600')
#observation_dimensions = 14*n_frames # 14 objects
#observation_dimensions = (size[0], size[1], size[2]*n_frames)
observation_dimensions = (100, 80, 3*n_frames)
num_actions = env.action_space.n
possible_actions = np.array(np.identity(num_actions,dtype=int).tolist())


print("The size of our frame is: ", env.observation_space)

# Here we create an hot encoded version of our actions
# possible_actions = [[1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]...]
possible_actions = np.array(np.identity(num_actions,dtype=int).tolist())


def mlp(x, sizes, activation=tf.tanh, output_activation=None):    
    # Build a feedforward neural network
    for size in sizes[:-1]:
        x = layers.Dense(units=size, activation=activation)(x)
    return layers.Dense(units=sizes[-1], activation=output_activation)(x)

# Train the policy by maxizing the PPO-Clip objective
@tf.function
def train_policy(
    observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
):

    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        ratio = tf.exp(
            logprobabilities(actor(observation_buffer), action_buffer)
            - logprobability_buffer
        )
        min_advantage = tf.where(
            advantage_buffer > 0,
            (1 + clip_ratio) * advantage_buffer,
            (1 - clip_ratio) * advantage_buffer,
        )

        policy_loss = -tf.reduce_mean(
            tf.minimum(ratio * advantage_buffer, min_advantage)
        )
    policy_grads = tape.gradient(policy_loss, actor.trainable_variables)
    policy_optimizer.apply_gradients(zip(policy_grads, actor.trainable_variables))

    kl = tf.reduce_mean(
        logprobability_buffer
        - logprobabilities(actor(observation_buffer), action_buffer)
    )
    kl = tf.reduce_sum(kl)
    return kl


# Train the value function by regression on mean-squared error
@tf.function
def train_value_function(observation_buffer, return_buffer):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        value_loss = tf.reduce_mean((return_buffer - critic(observation_buffer)) ** 2)
    value_grads = tape.gradient(value_loss, critic.trainable_variables)
    value_optimizer.apply_gradients(zip(value_grads, critic.trainable_variables))

# Topologia de convolucion basada en parte por resnet50 y Unet

observation_input = keras.Input(shape=(observation_dimensions), dtype=tf.float32)


c1 = conv_block(observation_input,(16,16,32))
c1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
c2 = conv_block(c1,(32,32,64),d=0.2)
c2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
c3 = conv_block(c2,(64,64,128),d=0.3)
c3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c3)
c3 = tf.keras.layers.GlobalAveragePooling2D()(c3)
"""
c1 = conv_block(observation_input,(8,8))
c1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
c2 = conv_block(c1,(16,16),d=0.2)
c2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
c3 = conv_block(c2,(32,32),d=0.3)
c3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c3)
c3 = tf.keras.layers.GlobalAveragePooling2D()(c3)
"""
# Initialize the actor and the critic as keras models
logits = mlp(c3, list(hidden_sizes) + [num_actions], tf.tanh, None)
actor = keras.Model(inputs=observation_input, outputs=logits)
value = mlp(c3, list(hidden_sizes) + [1], tf.tanh, None)

critic = keras.Model(inputs=observation_input, outputs=value)

# Initialize the policy and the value function optimizers
policy_optimizer = keras.optimizers.Adam(learning_rate=policy_learning_rate)
value_optimizer = keras.optimizers.Adam(learning_rate=value_function_learning_rate)

The size of our frame is:  Box(0, 255, (210, 160, 3), uint8)


In [26]:
actor.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 210, 160, 1  0           []                               
                                2)]                                                               
                                                                                                  
 conv2d_36 (Conv2D)             (None, 210, 160, 16  208         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 210, 160, 16  64         ['conv2d_36[0][0]']              
 ormalization)                  )                                                           

In [45]:
# Initialize the buffer
buffer = Buffer(observation_dimensions, steps_per_epoch)

# Initialize the observation, episode return and episode length
observation, episode_return, episode_length = env_reset(env), 0, 0

# Iterate over the number of epochs
for epoch in range(epochs):
    # Initialize the sum of the returns, lengths and number of episodes for each epoch
    sum_return = 0;   sum_length = 0;   num_episodes = 0;   episode_max_return = 0

    # Iterate over the steps of each epoch
    for t in range(steps_per_epoch):
        observation=observation.reshape((1,observation.shape[0],observation.shape[1],observation.shape[2]))
        # Get the logits, action, and take one step in the environment
        logits, action = sample_action(observation)
        observation_new, reward, done, _ = env_step(env, action[0].numpy(), iters=10)
        episode_return += reward
        episode_length += 1

        # Get the value and log-probability of the action
        value_t = critic(observation)
        logprobability_t = logprobabilities(logits, action)
        # Store obs, act, rew, v_t, logp_pi_t
        buffer.store(observation, action, reward, value_t, logprobability_t,)
        # Update the observation
        observation = observation_new
        # Finish trajectory if reached to a terminal state
        terminal = done
        if terminal or (t == steps_per_epoch - 1):
            last_value = 0 if done else critic(observation.reshape((1,observation.shape[0],observation.shape[1],observation.shape[2])))
            buffer.finish_trajectory(last_value)
            sum_return += episode_return
            sum_length += episode_length
            num_episodes += 1
            if episode_return > episode_max_return: episode_max_return=episode_return
            if np.random.rand() > ratio:
              observation, episode_return, episode_length = env_reset(env,130), 0, 0 
            else:
              observation, episode_return, episode_length = env_reset(env), 0, 0 
            
    # Get values from the buffer
    (
        observation_buffer,
        action_buffer,
        advantage_buffer,
        return_buffer,
        logprobability_buffer,
    ) = buffer.get()

    # Update the policy and implement early stopping using KL divergence
    for _ in range(train_policy_iterations):
        kl = train_policy(
            observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
        )
        if kl > 1.5 * target_kl:
            # Early Stopping
            break

    # Update the value function
    for _ in range(train_value_iterations):
        train_value_function(observation_buffer, return_buffer)

    # Print mean return and length for each epoch
    print(
        f" Epoch: {epoch + 1}. MeanReturn: {sum_return / num_episodes}. MeanLength: {sum_length / num_episodes}. MaxReturn: {episode_max_return}"
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


 Epoch: 1. MeanReturn: -8.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 2. MeanReturn: -12.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 3. MeanReturn: -17.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 4. MeanReturn: -16.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 5. MeanReturn: -4.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 6. MeanReturn: -22.0. MeanLength: 500.0. MaxReturn: 0
 Epoch: 7. MeanReturn: -20.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 8. MeanReturn: -9.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 9. MeanReturn: -15.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 10. MeanReturn: -13.0. MeanLength: 500.0. MaxReturn: 0
 Epoch: 11. MeanReturn: -16.0. MeanLength: 500.0. MaxReturn: 0
 Epoch: 12. MeanReturn: -11.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 13. MeanReturn: -5.0. MeanLength: 500.0. MaxReturn: 0
 Epoch: 14. MeanReturn: -17.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 15. MeanReturn: -25.5. MeanLength: 500.0. MaxReturn: 0
 Epoch: 16. MeanReturn: -26.5. MeanLength: 500.0. MaxReturn: 0
 Epoc

#Primera Simulacion Env reset con 130 iteraciones de entrada

### Simulación

In [49]:
episode_return=0.0
i=0
while episode_return < 2:
  observation, episode_return, episode_length = env_reset(env,10), 0, 0
  images = []
  for t in range(steps_per_epoch):
      # Get the logits, action, and take one step in the environment
      logits, action = sample_action(observation.reshape((1,observation.shape[0],observation.shape[1],observation.shape[2])))
      observation_new, reward, done, _ = env_step(env, action[0].numpy(), iters=10, show=True)

      episode_return += reward

      # Update the observation
      observation = observation_new

      # Finish trajectory if reached to a terminal state
      if done:
        observation = env_reset(env)
        print(episode_return)
        break
  i+=1
episode_return

-13.0
-14.0
-41.0
-29.0
-9.0
-7.0
-1.0
-32.0
-41.0
-11.0
0.0
-23.0
-7.0
-8.0
-13.0
2.0


2.0

In [38]:
episode_return=0.0
i=0
while episode_return < 1 and i<10:
  observation, episode_return, episode_length = env_reset(env), 0, 0
  images = []
  for t in range(steps_per_epoch):
      # Get the logits, action, and take one step in the environment
      logits, action = sample_action(observation.reshape((1,observation.shape[0],observation.shape[1],observation.shape[2])))
      observation_new, reward, done, _ = env_step(env, action[0].numpy(), iters=10, show=True)

      episode_return += reward

      # Update the observation
      observation = observation_new

      # Finish trajectory if reached to a terminal state
      if done:
        observation = env_reset(env)
        print(episode_return)
        break
  i+=1
episode_return

1.0

In [47]:
episode_return=0.0
i=0
while episode_return < 2:
  observation, episode_return, episode_length = env_reset(env,10), 0, 0
  images = []
  for t in range(steps_per_epoch):
      # Get the logits, action, and take one step in the environment
      logits, action = sample_action(observation.reshape((1,observation.shape[0],observation.shape[1],observation.shape[2])))
      observation_new, reward, done, _ = env_step(env, action[0].numpy(), iters=10, show=True)

      episode_return += reward

      # Update the observation
      observation = observation_new

      # Finish trajectory if reached to a terminal state
      if done:
        observation = env_reset(env)
        print(episode_return)
        break
  i+=1
episode_return

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


-13.0
0.0
-29.0
-37.0


KeyboardInterrupt: ignored

### Creación del video

In [ ]:
# Librearías requeridas
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

## Boxing v4 epocas y 1000 iteraciones ResNet

In [ ]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 10, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400,maxduration=120)

100%|█████████▉| 701/702 [00:00<00:00, 2337.08it/s]


In [39]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 10, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400,maxduration=120)

100%|█████████▉| 600/601 [00:00<00:00, 1618.51it/s]


In [43]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 10, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400,maxduration=120)

100%|█████████▉| 700/701 [00:00<00:00, 1693.94it/s]


In [ ]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 10, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400,maxduration=120)

#Segunda Simulacion

### Simulación

In [ ]:
episode_return=0.0
i=0
while episode_return < 1:
  observation, episode_return, episode_length = env_reset(env,10), 0, 0
  images = []
  for t in range(steps_per_epoch):
      # Get the logits, action, and take one step in the environment
      logits, action = sample_action(observation.reshape((1,observation.shape[0],observation.shape[1],observation.shape[2])))
      observation_new, reward, done, _ = env_step(env, action[0].numpy(), iters=10, show=True)

      episode_return += reward

      # Update the observation
      observation = observation_new

      # Finish trajectory if reached to a terminal state
      if done:
        observation = env_reset(env)
        print(episode_return)
        break
  i+=1
episode_return

### Creación del video

In [ ]:
# Librearías requeridas
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

## Boxing v4 10 epocas y 1000 iteraciones UNet

In [ ]:
import cv2
from moviepy.editor import *

def create_video(images, filename):
  res=(160,210) #resulotion
  out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc('M','J','P','G'), 10, res)
  for image in images:
      out.write(image)

  out.release()
  return VideoFileClip(filename)

clip=create_video(images, "tmp.avi")
clip.ipython_display(width=400,maxduration=120)

100%|█████████▉| 714/715 [00:00<00:00, 1669.28it/s]
